<a href="https://colab.research.google.com/github/mattstoebe/congressStockTrading/blob/main/00_Get_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandasql

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26800 sha256=792a795ac2e9acf8ea5c49b5ba516671483f3559cb48a2a2519ed01cc066d843
  Stored in directory: c:\users\mattl\appdata\local\pip\cache\wheels\68\5d\a5\edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql


In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from urllib.parse import urlparse, parse_qs
import math
import statistics

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the main page to navigate from
main_url = 'https://www.quiverquant.com/congresstrading'  # Main page URL

# Get Main page HTML
response = requests.get(main_url)

# Check if the request was successful
if response.status_code == 200:

    soup = BeautifulSoup(response.text, 'html.parser')
    # Find all tables with the class 'table-congress table-politician'
    tables = soup.find_all('table', class_='table-congress table-politician')

    # make sure two tables are found, and select the second table
    if len(tables) >= 2:
        second_table = tables[1]

        # Empty set to store unique URLs
        unique_urls = set()

        # Find all links in the second table
        links = second_table.find_all('a', href=True)

        # Add the unique href attributes (URLs) to the set
        for link in links:
            if '/congresstrading/politician' in link['href']:
                unique_urls.add(link['href'])

        # Convert the set to a list if needed
        urls = list(unique_urls)

        # URL list for testing
        #print(urls)

        # The rest of your scraping logic goes here...
        # Visit each URL and extract the required information

    else:
        print("Second table not found or there are fewer than two tables.")
else:
    print("Failed to retrieve the webpage")


In [ ]:
base_url = "https://www.quiverquant.com"
unique_redirect_urls = set()

for url in urls:
    full_url = base_url + url.replace('..', '')
    response = requests.get(full_url, allow_redirects=True)
    if response.status_code == 200:
        final_url = response.url  # Gets the final redirected URL
        # Optional: If you want to standardize the URL (e.g., removing query parameters)
        parsed_url = urlparse(final_url)
        clean_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
        unique_redirect_urls.add(clean_url)
    else:
        print(f"Failed to retrieve {full_url}")

In [ ]:
print(len(unique_redirect_urls))

72


In [ ]:
all_rows = []
run_urls = list(unique_redirect_urls)
for url in run_urls:
    if "politician" in url:
        #print(url)
        time.sleep(1)
        response = requests.get(url)  # Update response for each URL
        #print(response.status_code)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table', class_='table-congress table-politician')
            if table:
                for row in table.find_all('tr')[1:]:
                    # Initialize an empty dictionary for each row
                    row_data = {}

                    # Extract cells from the row
                    cells = row.find_all('td')

                    # Ticker, StockName, StockType
                    first_cell = cells[0].find('a')
                    spans = first_cell.find_all('span', recursive=False) if first_cell else []
                    row_data['Ticker'] = spans[0].get_text(strip=True) if spans else '-'
                    row_data['StockName'] = spans[1].get_text(strip=True) if len(spans) > 1 else '-'
                    row_data['StockType'] = spans[2].get_text(strip=True) if len(spans) > 2 else '-'

                    # Transaction data
                    transaction_cell = cells[1].find('a')
                    transaction_spans = transaction_cell.find_all('span', recursive=False) if transaction_cell else []
                    row_data['TransactionType'] = transaction_spans[0].get_text(strip=True) if transaction_spans else '-'
                    row_data['TransactionAmount'] = transaction_spans[1].get_text(strip=True) if len(transaction_spans) > 1 else '-'

                    # Politician data
                    politician_cell = cells[2].find('a')
                    politician_spans = politician_cell.find_all('span', recursive=False) if politician_cell else []
                    row_data['PoliticianName'] = politician_spans[0].get_text(strip=True) if politician_spans else '-'
                    row_data['PoliticianDetails'] = politician_spans[1].get_text(strip=True) if len(politician_spans) > 1 else '-'

                    # Filed, Traded, Description, Performance
                    row_data['Filed'] = cells[3].get_text(strip=True)
                    row_data['Traded'] = cells[4].get_text(strip=True)
                    row_data['Description'] = cells[5].get_text(strip=True)
                    row_data['Performance'] = cells[6].get_text(strip=True)
                    # Add the row data to the list
                    all_rows.append(row_data)

            else:
                print("Table not found")
        else:
            print("Failed to retrieve the webpage")


In [ ]:
df1 = pd.DataFrame(all_rows)

print(df1.shape)

df1.head()

#df1.to_csv(r'Downloads/congress_stock_Transactions_12_31.csv')

(20560, 11)


,Ticker,StockName,StockType,TransactionType,TransactionAmount,PoliticianName,PoliticianDetails,Filed,Traded,Description,Performance
0,ITW,Illinois Tool Works Inc. Common Stock,Stock,Sale (Full),"$15,001 - $50,000",Tammy Duckworth,Senate / D,"May. 01, 2023","Apr. 11, 2023",-,-2.43%
1,DIS,The Walt Disney Company,Stock,Sale (Full),"$1,001 - $15,000",Tammy Duckworth,Senate / D,"Apr. 16, 2021","Apr. 05, 2021",-,-69.06%
2,ITW,Illinois Tool Works Inc.,Stock,Sale (Partial),"$1,001 - $15,000",Tammy Duckworth,Senate / D,"Apr. 16, 2021","Apr. 05, 2021",-,0.87%
3,VZ,Verizon Communications Inc.,Stock,Sale (Full),"$15,001 - $50,000",Tammy Duckworth,Senate / D,"Apr. 16, 2021","Apr. 05, 2021",-,-52.92%
4,GE,General Electric Company,Stock,Sale (Full),"$1,001 - $15,000",Tammy Duckworth,Senate / D,"Oct. 26, 2018","Aug. 07, 2018",-,-45.25%


### Read from CSV to avoid ReScraping


In [ ]:
df = pd.read_csv(r'Downloads/congress_stock_Transactions_12_31.csv')

In [ ]:
# Create New Columns For Dates
df1 = df
df1["Traded"] = pd.to_datetime(df1["Traded"])
df1["Filed"] = pd.to_datetime(df1["Filed"], format = '%b. %d, %Y')
df1["Transaction_Year"] = df1["Traded"].dt.year
df1["Filing_Delay"] = df1["Filed"] - df1["Traded"]

df1.head()

,Unnamed: 0,Ticker,StockName,StockType,TransactionType,TransactionAmount,PoliticianName,PoliticianDetails,Filed,Traded,Description,Performance,Transaction_Year,Filing_Delay
0,0,ITW,Illinois Tool Works Inc. Common Stock,Stock,Sale (Full),"$15,001 - $50,000",Tammy Duckworth,Senate / D,2023-05-01,2023-04-11,-,-2.43%,2023,20 days
1,1,DIS,The Walt Disney Company,Stock,Sale (Full),"$1,001 - $15,000",Tammy Duckworth,Senate / D,2021-04-16,2021-04-05,-,-69.06%,2021,11 days
2,2,ITW,Illinois Tool Works Inc.,Stock,Sale (Partial),"$1,001 - $15,000",Tammy Duckworth,Senate / D,2021-04-16,2021-04-05,-,0.87%,2021,11 days
3,3,VZ,Verizon Communications Inc.,Stock,Sale (Full),"$15,001 - $50,000",Tammy Duckworth,Senate / D,2021-04-16,2021-04-05,-,-52.92%,2021,11 days
4,4,GE,General Electric Company,Stock,Sale (Full),"$1,001 - $15,000",Tammy Duckworth,Senate / D,2018-10-26,2018-08-07,-,-45.25%,2018,80 days


In [ ]:
# Clean up the TransactionAmount Data

# Split the "TransactionAmount" column into lists based on "-"
df1["TransactionAmount_Bounds"] = df1["TransactionAmount"].str.split("-")

# Use apply to remove the dollar sign ('$') and comma(',') from each element in the bounds column
df1["TransactionAmount_Bounds"] = df1["TransactionAmount_Bounds"].apply(lambda lst: [item.replace('$', '').replace(',', '') for item in lst])

#Connvert each element in the bounds column to a float
df1["TransactionAmount"] = df1["TransactionAmount_Bounds"].apply(lambda lst: [float(item) for item in lst])

df1["Avg_Trade_Value"] = df1["TransactionAmount"].apply(lambda lst: int(statistics.mean(lst)))

df1["Trade_bucket"] = df1["Avg_Trade_Value"].apply(lambda x: "Low" if x < 15000 else("Mid" if x < 50000 else "High"))

df1.drop(["Avg_Trade_Value", "TransactionAmount_Bounds", "Unnamed: 0"], axis = 1, inplace = True)

df1["Filed_Adjusted"] = df1.apply(
    lambda row: (
        row["Traded"] + pd.DateOffset(days=1)
        if row["Filed"] <= row["Traded"]
        else row["Filed"]
    ),
    axis=1,
)

df1.head()

,Ticker,StockName,StockType,TransactionType,TransactionAmount,PoliticianName,PoliticianDetails,Filed,Traded,Description,Performance,Transaction_Year,Filing_Delay,Trade_bucket,Filed_Adjusted
0,ITW,Illinois Tool Works Inc. Common Stock,Stock,Sale (Full),"[15001.0, 50000.0]",Tammy Duckworth,Senate / D,2023-05-01,2023-04-11,-,-2.43%,2023,20 days,Mid,2023-05-01
1,DIS,The Walt Disney Company,Stock,Sale (Full),"[1001.0, 15000.0]",Tammy Duckworth,Senate / D,2021-04-16,2021-04-05,-,-69.06%,2021,11 days,Low,2021-04-16
2,ITW,Illinois Tool Works Inc.,Stock,Sale (Partial),"[1001.0, 15000.0]",Tammy Duckworth,Senate / D,2021-04-16,2021-04-05,-,0.87%,2021,11 days,Low,2021-04-16
3,VZ,Verizon Communications Inc.,Stock,Sale (Full),"[15001.0, 50000.0]",Tammy Duckworth,Senate / D,2021-04-16,2021-04-05,-,-52.92%,2021,11 days,Mid,2021-04-16
4,GE,General Electric Company,Stock,Sale (Full),"[1001.0, 15000.0]",Tammy Duckworth,Senate / D,2018-10-26,2018-08-07,-,-45.25%,2018,80 days,Low,2018-10-26


In [ ]:
df1.to_csv(r'Downloads/congress_stock_Transactions_cleaned.csv')

In [4]:
#testing upload to github